In [17]:
import pickle

import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

import rental_utils; reload(rental_utils)

%matplotlib inline
sns.set(style="whitegrid")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\philipp.deutsch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_raw = rental_utils.clean(pd.read_json('Data/train.json'))
test_raw = rental_utils.clean(pd.read_json('Data/test.json'))

In [4]:
data = train_test_split(train_raw[['description', 'listing_id']], train_raw['interest_level'], test_size=0.33, random_state=1)
[X_train, X_val, y_train, y_val] = data

In [5]:
train_descriptions = rental_utils.convert_to_words(X_train)

 93% (30998 of 33065) |###################################################    | Elapsed Time: 0:01:30 ETA: 0:00:05D:\Analytics\Anaconda 2.1.0\envs\ds\lib\site-packages\bs4\__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
100% (33065 of 33065) |######################################################| Elapsed Time: 0:01:36 Time: 0:01:36


In [6]:
val_descriptions = rental_utils.convert_to_words(X_val)

100% (16287 of 16287) |######################################################| Elapsed Time: 0:00:49 Time: 0:00:49


In [7]:
test_descriptions = rental_utils.convert_to_words(test_raw)

 99% (74646 of 74659) |###################################################### | Elapsed Time: 0:03:42 ETA: 0:00:00


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word",
                                 tokenizer=None,
                                 preprocessor=None,
                                 stop_words=None,
                                 max_features=1000)
train_data_features = vectorizer.fit_transform(train_descriptions).toarray()
val_data_features = vectorizer.transform(val_descriptions).toarray()

In [19]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 1000) 
forest = forest.fit(train_data_features, y_train)

pred_val = forest.predict(val_data_features)
ct = pd.crosstab(pred_val, y_val, margins=False)
print(ct)

interest_level  high    low  medium
row_0                              
high             162     94     189
low              866  10579    2894
medium           226    597     680


In [20]:
test_data_features = vectorizer.transform(test_descriptions).toarray()

probs = forest.predict_proba(test_data_features)
probs = pd.DataFrame(probs, columns=forest.classes_)
probs.head()

,high,low,medium
0,0.055372,0.525526,0.419102
1,0.050167,0.788262,0.161571
2,0.050417,0.335458,0.614125
3,0.107242,0.528608,0.364150
4,0.004750,0.938167,0.057083


In [21]:
submission = test_raw[['listing_id']]
submission = pd.concat([submission.reset_index(drop=True),
                        pd.DataFrame(probs, columns=forest.classes_)],
                       axis=1)
submission = submission[['listing_id', 'high', 'medium', 'low']]
timestamp = str(datetime.datetime.now())[:16]
submission_name = 'Submissions/submission_desc_ ' + timestamp + '.csv'
submission_name = submission_name.replace(' ', '_').replace(':', '')
submission.to_csv(submission_name, index=False)